In [41]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 



url = "C:/Users/Nimrod/Documents/GitHub/2nd_try_iron_repo/week4/Data_Marketing_Customer_Analysis_Round3.csv"

df = pd.read_csv(url)

In [213]:
df

,region,customer_lifetime_value,response,coverage,education,effective_to_date,month,employment_status,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,central,4809,no,basic,college,2/18/11,feb,employed,m,48029,...,52,0,9,corporate auto,corporate l3,offer3,agent,292,four-door car,medsize
1,west region,2228,no,basic,college,1/18/11,jan,unemployed,f,92260,...,26,0,1,personal auto,personal l3,offer4,call center,744,four-door car,medsize
2,east,14947,no,basic,bachelor,2/10/11,feb,employed,m,22139,...,31,0,2,personal auto,personal l3,offer3,call center,480,suv,medsize
3,north west,22332,yes,extended,college,1/11/11,jan,employed,m,49078,...,3,0,2,corporate auto,corporate l3,offer2,branch,484,four-door car,medsize
4,north west,9025,no,premium,bachelor,1/17/11,jan,medical leave,f,23675,...,31,0,7,personal auto,personal l2,offer1,branch,707,four-door car,medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,central,15563,no,premium,bachelor,1/19/11,jan,unemployed,f,61541,...,40,0,7,personal auto,personal l1,offer3,web,1214,luxury car,medsize
10685,north west,5259,no,basic,college,1/6/11,jan,employed,f,61146,...,68,0,6,personal auto,personal l3,offer2,branch,273,four-door car,medsize
10686,central,23893,no,extended,bachelor,2/6/11,feb,employed,f,39837,...,63,0,2,corporate auto,corporate l3,offer1,web,381,luxury suv,medsize
10687,west region,11971,no,premium,college,2/13/11,feb,employed,f,64195,...,27,4,6,personal auto,personal l1,offer1,branch,618,suv,medsize


In [42]:
df_num = df.select_dtypes(include='number')

In [212]:
df_num.columns

Index(['customer_lifetime_value', 'income', 'monthly_premium_auto',
       'months_since_last_claim', 'months_since_policy_inception',
       'number_of_open_complaints', 'number_of_policies',
       'total_claim_amount'],
      dtype='object')

In [86]:
df_num = df_num.drop_duplicates()

df_num.isna().any()

customer_lifetime_value          False
income                           False
monthly_premium_auto             False
months_since_last_claim          False
months_since_policy_inception    False
number_of_open_complaints        False
number_of_policies               False
total_claim_amount               False
dtype: bool

## Spltting & Scalling

In [96]:
# splitting

X = df_num.drop(['total_claim_amount'], axis=1)
y = df_num[['total_claim_amount']]

In [198]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [199]:
# past split scaling 

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)



## Apply linear regression.


In [200]:
lm = LinearRegression()
model = lm.fit(X_train, y_train)



In [201]:
model.coef_


array([[-180.79796708, -111.099357  , 1352.90069037,   13.97004314,
          -7.18644081,    6.26408628,    4.71566173]])

In [202]:
model.intercept_

array([317.56674038])

## Model Interpretation

In [203]:
y_pred

,0
0,508.298322
1,493.451164
2,238.259211
3,320.242764
4,348.051610
5,897.763685
6,847.856196
7,275.319540
8,340.306945
9,513.722617


In [204]:
y_pred = model.predict(X_test)

y_pred = pd.DataFrame(y_pred)

In [205]:
y_test = y_test.reset_index(drop=True)

r_df = pd.concat([y_test, y_pred], axis=1) 

r_df = r_df.rename(columns={"total_claim_amount": "y_test", 0: "y_pred"})

In [207]:
r_df.isna().sum()

y_test    0
y_pred    0
dtype: int64

In [208]:
r_df.head(3)

,y_test,y_pred
0,631,514.793892
1,956,495.915276
2,271,231.360995


In [209]:
r_df['residual'] = r_df['y_test']-r_df['y_pred']
r_df.head()

,y_test,y_pred,residual
0,631,514.793892,116.206108
1,956,495.915276,460.084724
2,271,231.360995,39.639005
3,669,319.174191,349.825809
4,318,350.298236,-32.298236


## Model Interpretation

In [210]:
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse
import numpy as np

In [211]:
RMSE = np.sqrt(mse(y_test, y_pred))
MAE = mae(y_test, y_pred)

print(f'\nModel Interpretation\n{"*"*25}\nMAE:{MAE}\nRMSE: {RMSE}\n')


Model Interpretation
*************************
MAE:163.1006739680857
RMSE: 235.8458033341696

